In [1]:
# -*- coding: utf-8 -*-
"""
Skrip untuk Model Rekomendasi Proyek KPBU untuk Investor

Notebook ini mengimplementasikan sistem rekomendasi hibrida untuk mencocokkan
profil investor dengan proyek yang tersedia berdasarkan data proyek yang sudah ada.
Versi ini interaktif dan meminta pengguna untuk mengisi profil mereka.
"""

# ---------------------------------------------------------------------------
# Langkah 1: Impor Pustaka dan Muat Data Proyek
# ---------------------------------------------------------------------------
import pandas as pd
import numpy as np
import io
from google.colab import files

# Minta pengguna untuk mengunggah file CSV dataset proyek
# Gunakan file yang sama dengan yang Anda gunakan untuk melatih model pertama
print("Silakan unggah file CSV dataset proyek Anda (misal: 'data_proyek_kpbu_realistis.csv').")
uploaded = files.upload()

try:
    file_name = next(iter(uploaded))
    # Muat dataset proyek ke dalam DataFrame
    df_proyek = pd.read_csv(io.BytesIO(uploaded[file_name]))
    print(f"\nFile '{file_name}' berhasil diunggah. Terdapat {len(df_proyek)} proyek dalam database.")
except (StopIteration, KeyError):
    print("\nTidak ada file yang diunggah. Silakan jalankan sel ini lagi.")
    # Hentikan eksekusi jika tidak ada file
    raise SystemExit()


Silakan unggah file CSV dataset proyek Anda (misal: 'data_proyek_kpbu_realistis.csv').


Saving dummydata_kpbu.csv to dummydata_kpbu.csv

File 'dummydata_kpbu.csv' berhasil diunggah. Terdapat 100 proyek dalam database.


In [2]:
# ---------------------------------------------------------------------------
# Langkah 2: Dapatkan Profil Investor Secara Interaktif
# ---------------------------------------------------------------------------

def get_investor_profile_interactive(df):
    """Fungsi untuk mendapatkan profil investor dari input pengguna secara interaktif."""

    print("\n--- Kuesioner Profil Investor ---")
    print("Silakan isi profil Anda untuk mendapatkan rekomendasi proyek.")

    # Inisialisasi profil
    profile = {}

    # 1. Toleransi Risiko
    risk_options = {'1': 'Konservatif', '2': 'Moderat', '3': 'Agresif'}
    while True:
        print("\n1. Apa toleransi risiko Anda?")
        for key, value in risk_options.items():
            print(f"   {key}: {value}")
        choice = input("   Pilih nomor (1/2/3): ")
        if choice in risk_options:
            profile['toleransi_risiko'] = risk_options[choice]
            break
        else:
            print("Pilihan tidak valid, silakan coba lagi.")

    # 2. Preferensi Sektor
    sektor_options = sorted(df['Sektor_Proyek'].unique())
    while True:
        print("\n2. Sektor apa yang Anda minati? (Bisa pilih lebih dari satu)")
        for i, sektor in enumerate(sektor_options, 1):
            print(f"   {i}: {sektor}")
        print("   Masukkan nomor sektor yang Anda inginkan, pisahkan dengan koma (contoh: 1, 3, 4)")
        choice_str = input("   Pilihan Anda: ")
        try:
            chosen_indices = [int(i.strip()) - 1 for i in choice_str.split(',')]
            if all(0 <= idx < len(sektor_options) for idx in chosen_indices):
                profile['preferensi_sektor'] = [sektor_options[idx] for idx in chosen_indices]
                break
            else:
                 print("Satu atau lebih nomor pilihan tidak valid, silakan coba lagi.")
        except ValueError:
            print("Format input salah. Harap masukkan nomor yang dipisahkan koma.")

    # 3. Horison Investasi
    horizon_options = {'1': 'Jangka Pendek', '2': 'Jangka Menengah', '3': 'Jangka Panjang'}
    while True:
        print("\n3. Apa horison investasi Anda?")
        for key, value in horizon_options.items():
            print(f"   {key}: {value} (<5 thn, 5-15 thn, >15 thn)")
        choice = input("   Pilih nomor (1/2/3): ")
        if choice in horizon_options:
            profile['horison_investasi'] = horizon_options[choice]
            break
        else:
            print("Pilihan tidak valid, silakan coba lagi.")

    return profile

# Panggil fungsi interaktif untuk mendapatkan profil investor
investor_profile = get_investor_profile_interactive(df_proyek)

print("\n--- Profil Investor Anda Telah Disimpan ---")
print(f"Toleransi Risiko: {investor_profile['toleransi_risiko']}")
print(f"Preferensi Sektor: {', '.join(investor_profile['preferensi_sektor'])}")
print(f"Horison Investasi: {investor_profile['horison_investasi']}")


--- Kuesioner Profil Investor ---
Silakan isi profil Anda untuk mendapatkan rekomendasi proyek.

1. Apa toleransi risiko Anda?
   1: Konservatif
   2: Moderat
   3: Agresif
   Pilih nomor (1/2/3): 2

2. Sektor apa yang Anda minati? (Bisa pilih lebih dari satu)
   1: Air & Sanitasi
   2: Energi
   3: Jalan & Jembatan
   4: Kesehatan
   5: Pendidikan
   6: Telekomunikasi
   7: Transportasi
   Masukkan nomor sektor yang Anda inginkan, pisahkan dengan koma (contoh: 1, 3, 4)
   Pilihan Anda: 1,2,5

3. Apa horison investasi Anda?
   1: Jangka Pendek (<5 thn, 5-15 thn, >15 thn)
   2: Jangka Menengah (<5 thn, 5-15 thn, >15 thn)
   3: Jangka Panjang (<5 thn, 5-15 thn, >15 thn)
   Pilih nomor (1/2/3): 1

--- Profil Investor Anda Telah Disimpan ---
Toleransi Risiko: Moderat
Preferensi Sektor: Air & Sanitasi, Energi, Pendidikan
Horison Investasi: Jangka Pendek


In [3]:
# ---------------------------------------------------------------------------
# Langkah 3: Implementasi Matching Engine
# ---------------------------------------------------------------------------

# --- Bagian A: Filter Keras (Rule-Based Filtering) ---
def filter_proyek_berdasarkan_risiko(df, profil_investor):
    """Fungsi untuk menyaring proyek berdasarkan toleransi risiko investor."""
    toleransi = profil_investor['toleransi_risiko']

    if toleransi == 'Konservatif':
        # Hanya proyek dengan risiko 'Rendah'
        allowed_risks = ['Rendah']
    elif toleransi == 'Moderat':
        # Proyek dengan risiko 'Rendah' dan 'Menengah'
        allowed_risks = ['Rendah', 'Menengah']
    elif toleransi == 'Agresif':
        # Semua proyek diizinkan
        return df
    else:
        # Default jika input tidak valid
        return pd.DataFrame()

    # Lakukan filter
    filtered_df = df[df['Profil_Risiko'].isin(allowed_risks)].copy()
    return filtered_df

# --- Bagian B: Skoring Kecocokan (Content-Based Scoring) ---
def hitung_skor_kecocokan(proyek, profil_investor):
    """Fungsi untuk menghitung skor kecocokan sebuah proyek dengan profil investor."""

    # Definisikan bobot untuk setiap kriteria
    bobot = {
        'sektor': 0.5,
        'horison': 0.5
    }

    skor_akhir = 0

    # 1. Hitung Skor Kecocokan Sektor
    skor_sektor = 0
    if proyek['Sektor_Proyek'] in profil_investor['preferensi_sektor']:
        skor_sektor = 1.0

    # 2. Hitung Skor Kecocokan Horison Investasi
    skor_horison = 0
    durasi_proyek = proyek['Durasi_Konsesi_Tahun']
    horison_investor = profil_investor['horison_investasi']

    if horison_investor == 'Jangka Pendek' and durasi_proyek <= 5:
        skor_horison = 1.0
    elif horison_investor == 'Jangka Menengah' and 5 < durasi_proyek <= 15:
        skor_horison = 1.0
    elif horison_investor == 'Jangka Panjang' and durasi_proyek > 15:
        skor_horison = 1.0
    else:
        # Beri skor parsial jika tidak cocok persis
        skor_horison = 0.2

    # 3. Hitung Skor Total dengan Pembobotan
    skor_akhir = (bobot['sektor'] * skor_sektor) + (bobot['horison'] * skor_horison)

    return skor_akhir

In [4]:
# ---------------------------------------------------------------------------
# Langkah 4: Proses Pencocokan dan Hasilkan Rekomendasi
# ---------------------------------------------------------------------------

print("\n--- Memulai Proses Pencocokan ---")

# 1. Terapkan Filter Keras
proyek_tersaring = filter_proyek_berdasarkan_risiko(df_proyek, investor_profile)
print(f"Langkah 1: Ditemukan {len(proyek_tersaring)} proyek yang sesuai dengan toleransi risiko '{investor_profile['toleransi_risiko']}'.")

if not proyek_tersaring.empty:
    # 2. Hitung Skor Kecocokan untuk setiap proyek yang lolos filter
    proyek_tersaring['skor_kecocokan'] = proyek_tersaring.apply(
        lambda row: hitung_skor_kecocokan(row, investor_profile),
        axis=1
    )
    print("Langkah 2: Menghitung skor kecocokan untuk setiap proyek...")

    # 3. Urutkan proyek berdasarkan skor tertinggi
    rekomendasi_final = proyek_tersaring.sort_values(by='skor_kecocokan', ascending=False)

    # 4. Ambil Top 5 Rekomendasi
    top_5_rekomendasi = rekomendasi_final.head(5)

    print("Langkah 3: Mengurutkan proyek dan mengambil 5 rekomendasi teratas.")
else:
    top_5_rekomendasi = pd.DataFrame()



--- Memulai Proses Pencocokan ---
Langkah 1: Ditemukan 61 proyek yang sesuai dengan toleransi risiko 'Moderat'.
Langkah 2: Menghitung skor kecocokan untuk setiap proyek...
Langkah 3: Mengurutkan proyek dan mengambil 5 rekomendasi teratas.


In [5]:
# ---------------------------------------------------------------------------
# Langkah 5: Tampilkan Hasil Rekomendasi
# ---------------------------------------------------------------------------

print("\n=======================================================")
print("          HASIL REKOMENDASI PROYEK")
print("=======================================================")

if not top_5_rekomendasi.empty:
    # Atur kolom yang ingin ditampilkan
    kolom_tampil = [
        'Nama_Proyek',
        'Sektor_Proyek',
        'Profil_Risiko',
        'Durasi_Konsesi_Tahun',
        'skor_kecocokan'
    ]

    # Ubah format skor menjadi persentase untuk tampilan
    top_5_rekomendasi['skor_kecocokan'] = top_5_rekomendasi['skor_kecocokan'].map('{:.2%}'.format)

    print(top_5_rekomendasi[kolom_tampil].to_string(index=False))
else:
    print("Mohon maaf, tidak ada proyek yang cocok dengan profil Anda saat ini.")

print("=======================================================")


          HASIL REKOMENDASI PROYEK
                        Nama_Proyek  Sektor_Proyek Profil_Risiko  Durasi_Konsesi_Tahun skor_kecocokan
Asrama Mahasiswa Nusantara Surabaya     Pendidikan      Menengah                    31         60.00%
 Digitalisasi Perpustakaan Nasional     Pendidikan      Menengah                    14         60.00%
 IPA Kapasitas 500L/detik Pontianak Air & Sanitasi      Menengah                    26         60.00%
        PLT Panas Bumi Dieng Unit 2         Energi      Menengah                    21         60.00%
            PLTS Terapung Singkarak         Energi      Menengah                    21         60.00%


/tmp/ipython-input-5-991102166.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_5_rekomendasi['skor_kecocokan'] = top_5_rekomendasi['skor_kecocokan'].map('{:.2%}'.format)
